In [1]:
from selenium.webdriver import ChromiumEdge, EdgeOptions
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC

In [2]:
import time

In [3]:
import httpx

In [4]:
options = EdgeOptions()
prefs = {
    "browser": {"show_hub_popup_on_download_start": False},
    "user_experience_metrics": {"personalization_data_consent_enabled": True},
}
options.add_experimental_option("prefs", prefs)

In [5]:
driver = ChromiumEdge(options=options)

In [6]:
driver.implicitly_wait(1)

In [7]:
driver.get("https://digital.feprecisionplus.com/greateasternlife")

In [8]:
WebDriverWait(driver, 60).until(
    EC.invisibility_of_element(("id", "loading-indicator-undefined-overlay"))
)
time.sleep(1)
driver.find_element("id", "lblchkCategoryId-1-1").click()

In [9]:
WebDriverWait(driver, 60).until(
    EC.invisibility_of_element(("id", "loading-indicator-undefined-overlay"))
)
time.sleep(1)

In [10]:
funds = []
for i in range(4):
    WebDriverWait(driver, 60).until(
        EC.invisibility_of_element(("id", "loading-indicator-undefined-overlay"))
    )
    time.sleep(1)
    download_icons = driver.find_elements("class name", "fe-download-icon")
    visible_icons = [
        download_icon
        for download_icon in download_icons
        if download_icon.is_displayed()
    ]
    for visible_icon in visible_icons:
        visible_icon.click()
        time.sleep(0.5)
        funds.append(
            (
                visible_icon.get_attribute("data-code"),
                driver.find_element(
                    "xpath", '//*[@id="pricefundOptions"]/div/span[2]/form/div/a'
                ).text.split("\n")[0],
            )
        )
        driver.find_element("xpath", "/html/body/div[9]/div[1]/button").click()
    WebDriverWait(driver, 60).until(
        EC.invisibility_of_element(("id", "loading-indicator-undefined-overlay"))
    )
    if i < 3:
        driver.find_element(
            "xpath",
            "/html/body/div[6]/div[1]/div[1]/div/div[1]/div[3]/div[4]/table/tfoot/tr/td/div/div",
        ).find_element("class name", "ui-paging-next").click()

In [11]:
WebDriverWait(driver, 60).until(
    EC.invisibility_of_element(("id", "loading-indicator-undefined-overlay"))
)
time.sleep(1)
driver.find_element("id", "lblchkBespokeColumn5-1-2").click()
WebDriverWait(driver, 60).until(
    EC.invisibility_of_element(("id", "loading-indicator-undefined-overlay"))
)
time.sleep(1)

In [12]:
funds_with_dividends = []
download_icons = driver.find_elements("class name", "fe-download-icon")
visible_icons = [
    download_icon for download_icon in download_icons if download_icon.is_displayed()
]
for visible_icon in visible_icons:
    visible_icon.click()
    time.sleep(0.5)
    funds_with_dividends.append(
        (
            visible_icon.get_attribute("data-code"),
            driver.find_element(
                "xpath", '//*[@id="pricefundOptions"]/div/span[2]/form/div/a'
            ).text.split("\n")[0],
        )
    )
    driver.find_element("xpath", "/html/body/div[9]/div[1]/button").click()
WebDriverWait(driver, 60).until(
    EC.invisibility_of_element(("id", "loading-indicator-undefined-overlay"))
)

True

In [13]:
driver.quit()

In [14]:
for fund_code, fund_name in funds:
    res = httpx.get(
        "https://digital.feprecisionplus.com/greateasternlife/en-GB/GreatEastern_V2/DownloadTool/GetPriceHistoryFile",
        params={
            "modelString": {
                "GrsProjectId": "95400076",
                "ProjectName": "greateasternlife",
                "ToolId": 16,
                "LanguageId": "1",
                "LanguageCode": "en-GB",
                "forSaleIn": "",
                "FSIexclCT": "",
            },
            "filtersString": {
                "CitiCode": fund_code,
                "Universe": "SI",
                "TypeCode": f"FSI:{fund_code}",
                "FundName": fund_name,
                "BaseCurrency": "SGD",
                "PriceType": 2,
                "TimePeriod": "",
            },
        },
        timeout=30,
    )
    if res.is_success:
        with open(
            f"data/GreatLink/{res.headers['Content-Disposition'].split('filename=')[1].split('"')[1].split('_')[0]}.xlsx",
            "wb",
        ) as f:
            f.write(res.content)
    else:
        print(
            f"Failed to download {fund_name} {fund_code}: {res.status_code} {res.text}"
        )

In [15]:
for fund_code, fund_name in funds_with_dividends:
    res = httpx.get(
        "https://digital.feprecisionplus.com/greateasternlife/en-GB/GreatEastern_V2/DownloadTool/GetDividendHistoryFile",
        params={
            "modelString": {
                "GrsProjectId": "95400076",
                "ProjectName": "greateasternlife",
                "ToolId": 16,
                "LanguageId": "1",
                "LanguageCode": "en-GB",
                "forSaleIn": "",
                "FSIexclCT": "",
            },
            "filtersString": {
                "TypeCode": f"FSI:{fund_code}",
                "FundName": fund_name,
                "BaseCurrency": "SGD",
                "PriceType": 2,
                "TimePeriod": "",
            },
        },
        timeout=30,
    )
    if res.is_success:
        with open(
            f"data/GreatLink/{res.headers['Content-Disposition'].split('filename=')[1].split('"')[1].split('_')[0]}_Dividends.xlsx",
            "wb",
        ) as f:
            f.write(res.content)
    else:
        print(
            f"Failed to download {fund_name} {fund_code}: {res.status_code} {res.text}"
        )